### Log Reader  

In [ ]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="gemma3:1b", base_url="http://localhost:11434")

### Read the log file

In [ ]:
import os
from langchain.tools import tool
from langchain.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

@tool
def summarize_log() -> str:
    """Read and return summary of the first few lines from each .log file. DON'T get into details of them."""
    log_dir = './logs' 
    db_path = "./log_db"
    all_logs = []

    for file in os.listdir(log_dir):
        with open(os.path.join(log_dir, file)) as f:
            all_logs.extend([f"{file}: {line.strip()}" for line in f.readlines()[:50]]) 
    summary = "\n".join(all_logs) or "No log files found"
    
    # Check if log directory exists
    if not os.path.exists(db_path) and os.path.isdir(db_path):
        # splitting
        splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        chunks = splitter.create_documents([summary])    
        
        # embedding
        embedding = OllamaEmbeddings(model="nomic-embed-text")
        db = Chroma.from_documents(chunks, embedding, persist_directory=db_path)
        db.persist()
        summary += "\n\n[Embedding] DB already exists"
    else:
        summary += "\n\n[Embedding] No DB found (skipped creation)"
    
    return summary

In [ ]:
from langchain.agents import initialize_agent, AgentType
from langchain.tools import tool

# Define tools
@tool
def add_number(a: int, b: int) -> int:
    """Add two numbers and return the result"""
    return int(a) + int(b)

@tool
def substract_number(a: int, b: int) -> int:
    """Subtract two numbers and return the result"""
    return int(a) - int(b)

@tool
def multiply_number(a: int, b: int) -> int:
    """Multiply two numbers and return the result"""
    return int(a) * int(b)

# Register tools
tools = [add_number, substract_number, multiply_number, summarize_log] 

# Create agent (use structured chat to support multi-input tools)
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [ ]:
tool_by_name = {tool.name: tool for tool in tools}
tool_by_name

In [ ]:
#query = "What is the sum of 20 and 40"
query = "What are the errors in my logs?"
#query = "Hown WudfCoInstaller are in log file?"

results = agent.run(query)
print(results)